In [1]:
import random
from random import seed
import numpy as np

#for showing pics from files
%matplotlib inline
from matplotlib import pyplot
from matplotlib.image import imread

#for standarizing picture sizes
from os import listdir, makedirs
from numpy import asarray, save
from keras.preprocessing.image import load_img, img_to_array
from shutil import copyfile

#baseline model
import sys
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

KeyboardInterrupt: 

In [ ]:
# plot dog photos from the dogs vs cats dataset
# https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/

# define location of dataset
folder = 'train/'
# plot first few images
for i in range(9):
    # define subplot
    pyplot.subplot(330 + 1 + i)
    # define filename
    filename = folder + 'cat.' + str(i) + '.jpg'
    # load image pixels
    image = imread(filename)
    # plot raw pixel data
    pyplot.imshow(image)
# show the figure
pyplot.show()

In [ ]:
#standarizing photo sizes in training dataset to 200x200
#code to load the images into memory and resize
#save photoes for modeling
photos, labels = list(), list()

#enumerate files in the directory
for file in listdir(folder):
    #determine class (cat vs dog)
    output = 0.0
    
    if file.startswith('cat'):
        output = 1.0
        
    #load image
    photo = load_img(folder + file, target_size=(200,200))
    #convert to numpy array
    photo = img_to_array(photo)
    #store
    photos.append(photo)
    labels.append(output)
    
#convert to numpy arrays
photos = asarray(photos)
labels = asarray(labels)

print(photos.shape, labels.shape)

#save reshaped photos
save('dogs_vs_cats_photos.npy', photos)
save('dogs_vs_cats_labels.npy', labels)

In [ ]:
for i in range(9):
    # define subplot
    pyplot.subplot(330 + 1 + i)
    # define filename
    filename = folder + 'cat.' + str(i) + '.jpg'
    # load image pixels
    image = imread(filename)
    # plot raw pixel data
    print(labels[i])
    pyplot.imshow(image)
    
pyplot.show()


In [ ]:
#putting photos into standard directories
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']

for subdir in subdirs:
    #create label SUBdirectories
    labeldirs = ['dogs/', 'cats/']
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        
        #create directories in Python
        makedirs(newdir, exist_ok=True)

In [ ]:
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.45
# copy training dataset images into subdirectories
src_directory = 'train/'
for file in listdir(src_directory):
    src = src_directory + '/' + file
    dst_dir = 'train/'
    if random.random() < val_ratio: #splitting into test/train
        dst_dir = 'test/'
    if file.startswith('cat'):
        dst = dataset_home + dst_dir + 'cats/'  + file
        copyfile(src, dst)
    elif file.startswith('dog'):
        dst = dataset_home + dst_dir + 'dogs/'  + file
        copyfile(src, dst)

In [ ]:
#convolutional neural network model w/ 1 vgg style block
def define_model():

    model = Sequential() 
    
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')

    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')

    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()  
    
   # pyplot.show()
    

In [ ]:
#define model
def run_test_harness():
    model = define_model()
    
    train_datagen = ImageDataGenerator(rescale=1.0/255.0, width_shift_range=0.3, height_shift_range=0.3, zoom_range=0.2, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    
    train_it = train_datagen.flow_from_directory('dataset_dogs_vs_cats/train/',
        class_mode='binary', batch_size=64, target_size=(200, 200))
    test_it = test_datagen.flow_from_directory('dataset_dogs_vs_cats/test/',
        class_mode='binary', batch_size=64, target_size=(200, 200))
    
    
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs = 20, verbose=0)

    # save model
    model.save('final_model.h5')
    
    # evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
    print('> %.3f' % (acc * 100.0))
    
    # learning curves
    summarize_diagnostics(history)

In [ ]:
# entry point, run the test harness
run_test_harness()

In [ ]:
# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(200, 200))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 200, 200, 3)
    return img

In [ ]:
def run_example():
    
    # load the image
    folder = "test1/"
    for i in range(1, 2):
        # define subplot
        #pyplot.subplot(330 + 1 + i)
        # define filename
        filename = folder + str(i) + '.jpg'
        # load image pixels
        image = imread(filename)
        # plot raw pixel data
        
        
        img = load_image(filename)
        model = load_model('final_model.h5')
        result = model.predict(img)
        
        pyplot.imshow(image)
        
        if result[0] == 0.:
            title = "cat"
        else:
            title = "dog"
        
        pyplot.title(title)
        pyplot.show()
        
        #verification = input("Is this prediction correct?")
   
    
        
  

In [ ]:
#def validation():
    